# Clase 1 - Introducción a SCIP
## Investigación Operativa 2C 2025 - Departamento de Matemática - FCEN-UBA

### Inicialización

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [ ]:
!conda install -q pyscipopt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pyscipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-asl-1.0.0             |       h5888daf_2         504 KB  conda-forge
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-25.7.0               |  py312h7900ff3_0         1.2 MB  conda-forge
    cppad-20250000.2           |       h5888daf_0         487 KB  conda-forge
    gmp-6.3.0                  |       hac33072_2         449 KB  conda-forge
    ipopt-3.14.19              |       h0804adb_0        1000 KB  conda-forge
    libblas-3.9.0              |34_h59b9bed_openblas          19 KB  conda-forge
    libcblas-3.9.0             |34_he106b2a_o

In [ ]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model
# Importamos numpy
import numpy as np

### Resolviendo nuestro primer modelo

Vamos a resolver el modelo del ejemplo de producción de pintura utilizando SCIP

$$
\begin{array}{rrl}
        \max & 5x_1+4x_2 & \\
        \text{s.a.:} &  6x_1+4x_2 & \leq 24 \\
            & x_1 + 2x_2  & \leq 6 \\
            & x_1&  \leq  2 \\
            & x_2  & \leq x_1+1\\
            & x_1 & \geq 0 \\
            & x_2 & \geq 0 \\
            & x_1, x_2 & \in\mathbb{R}
    \end{array}
$$

Comenzamos inicializando el modelo con un nombre

In [ ]:
model = Model('Pintura')

Agregamos las variables $x_1$ y $x_2$ con el método ``addVar``. El método ``addVar`` admite dos argumentos:
- ``name`` : nombre de la variable
- ``vtype`` : tipo de variable, que puede ser continua (``'C'``), entera (``'I'``) o binaria (``'B'``) [Por defecto, ``vtype='C'``]
- ``ub`` : cota superior de la variable [Por defecto, ``ub=None``]
- ``lb`` : cota inferior para la variable [Por defecto, ``lb=0``]

In [ ]:
# Como nuestras dos variables son reales no negativas, solo hace falta especificar el nombre.
# (La cota superior de x1 la podemos agregar despues como una restriccion)
x1 = model.addVar(name='x1')
x2 = model.addVar(name='x2')

Ahora agregamos las restricciones. Para esto, utilizamos el método ``addCons`` que tiene a la restricción como argumento obligatorio y, opcionalmente, puede llevar un nombre.

Comenzamos con:
$$6x_1+4x_2 \leq 24$$

In [ ]:
model.addCons(6*x1 + 4*x2 <= 24, name='Cantidad de M1 disponible')

Cantidad de M1 disponible

Seguimos con:
$$x_1 + 2x_2  \leq 6$$

In [ ]:
model.addCons(x1 + 2*x2 <= 6, name='Cantidad de M2 disponible')

Cantidad de M2 disponible

Y, finalmente, agregamos las restricciones sobre la demanda:
$$x_1 \leq 2$$
$$x_2 \leq x_1 + 1$$

In [ ]:
model.addCons(x1 <= 2, name='Demanda PI')
model.addCons(x2 <= x1 + 1, name='Demanda PE')

Demanda PE

Finalmente, agregamos la función objetivo con el método ``setObjective``. Tiene un argumento obligatorio que es la expresi'on de la funci'on objetivo y el argumento ``sense`` que recibe ``minimize`` si queremos minimizar o ``maximize`` si queremos maximizar. Por defecto, ``sense=minimize``.

In [ ]:
model.setObjective(5*x1 + 4*x2, sense='maximize')

Ahora le pedimos a SCIP que resuelva el modelo, utilizando el método ``optimize``

In [ ]:
model.redirectOutput()
model.optimize()

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 2 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       0 del vars, 2 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 4, exhaustive) 1 del vars, 2 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       1 del vars, 4 del conss, 0 add conss, 6 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 2 medium, 2 exhaustive):
 2 deleted vars, 4 deleted constraints, 0 added constraints, 6 tightened bounds, 0 added holes, 0 changed sides, 

Obtenemos la solución mediante el método ``getBestSol``, que devuelve un diccionario ``{variable: valor_en_solucion}``

In [ ]:
sol = model.getBestSol()

Accedamos al valor de cada variable en la solución:

In [ ]:
for var in [x1, x2]:
  print(f'Valor de {var.name}: ', sol[var])

Valor de x1:  2.0
Valor de x2:  2.0


Accedamos al valor óptimo de la función objetivo con ``getObjVal``

In [ ]:
model.getObjVal()

18.0

Todo este procedimiento lo podemos llevar a cabo en una celda:

In [ ]:
# Inicializar modelo
model = Model('Pintura')

# Variables
x1 = model.addVar(name='x1')
x2 = model.addVar(name='x2')

# Restricciones
model.addCons(6*x1 + 4*x2 <= 24, name='Cantidad de M1 disponible')
model.addCons(x1 + 2*x2 <= 6, name='Cantidad de M2 disponible')
model.addCons(x1 <= 2, name='Demanda PI')
model.addCons(x2 <= x1 + 1, name='Demanda PE')

# Funcion objetivo
model.setObjective(5*x1 + 4*x2, sense='maximize')

# Optimizar
model.redirectOutput()
model.optimize()

sol = model.getBestSol()

# Solucion optima y valor optimo de la f.o.
for var in [x1, x2]:
  print(f'Valor de {var.name}: ', sol[var])

print(f'Valor optimo de la f.o.: {model.getObjVal()}')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 2 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       0 del vars, 2 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 4, exhaustive) 1 del vars, 2 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       1 del vars, 4 del conss, 0 add conss, 6 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 2 medium, 2 exhaustive):
 2 deleted vars, 4 deleted constraints, 0 added constraints, 6 tightened bounds, 0 added holes, 0 changed sides, 

Podemos guardar el modelo en un archivo con el método ``writeProblem`` al cual le indicamos el nombre del archivo y la extensión:

In [ ]:
model.writeProblem('pintura.cip')

wrote problem to file /content/pintura.cip


Para leer un modelo desde un archivo, utilizamos ``readProblem`` que toma como argumento el nombre del archivo donde se encuentra el modelo:

In [ ]:
model_2 = Model()
model_2.readProblem(filename='pintura.cip')
model_2.optimize()
sol_2 = model_2.getBestSol()
sol_2

Podemos guardar la solución con el método ``writeSol`` el cual toma como argumentos:
- la solucion del modelo
- ``filename`` : el nombre del archivo con el que queremos guardar la solucion [opcional]
- ``write_zeros`` : un booleano para indicar si escribe el valor de las variables cuyo valor en la solución es 0 (por defecto, ``write_zeros=False``)

In [ ]:
model.writeSol(sol, filename='solucion_pintura.sol')

### Optimizando ganancias de MicroApple

Vamos a resolver el modelo que busca la planificación óptima de adquisición de empresas para MicroApple:

$$\begin{array}{cll}
\max & \displaystyle \sum_{i=1}^N \sum_{k=1}^{12} g_{ik}x_{ik} &\\
\text{s.a:} & \displaystyle \sum_{k=1}^{12} x_{ik}  \leq 1 \quad \forall i\in \{1,\dots, N\} & \\
& \displaystyle \sum_{i=1}^{N} p_{ik}x_{ik}  \leq d_{k} \quad \forall k\in \{1,\dots,12\} & \\
& \displaystyle \sum_{k=1}^{12} \sum_{i\in E_s} x_{ik} \leq m_s \quad \forall s\in \{1,\dots, S\} & \\
 & \displaystyle \sum_{k=1}^{12} x_{5k} + x_{7k} + x_{11k} \leq 1 \quad & \\
& x_{ik}\in\{0,1\} \quad \forall i\in\{1,\dots,N\}\, \forall k\in \{1,\dots,12\} & \\
\end{array}$$

**RECORDAR**: los índices en Python comienzan con 0. Así, por ejemplo, podemos considerar que el mes 0 corresponde a enero y el mes 11 a diciembre.

Resolveremos una instancia con los siguientes parámetros:

In [ ]:
# Cantidad de potenciales empresas a ser adquiridas
N = 20

# Cantidad de rubros
S = 5

# Que empresas pertenecen a cada rubro
E = {0: [5, 18, 19],
    1: [7, 12],
    2: [2, 6, 10, 11, 16, 17],
    3: [1, 4, 8, 9, 13],
    4: [0, 3, 14, 15]}

# Maxima cantidad de cada rubro que puede ser adquirida
m = np.array([3, 1, 4, 4, 3])

# Presupuesto para cada mes
d = np.array([100, 120, 90, 50, 75, 60, 110, 85, 90, 105, 100, 50])

# Precio de cada empresa a lo largo del año (cada fila le corresponde a una empresa)
p = np.array([[79, 76, 76, 71, 69, 69, 59, 57, 53, 51, 47, 39],
              [81, 68, 68, 63, 62, 59, 58, 53, 42, 41, 40, 33],
              [84, 73, 70, 64, 50, 49, 48, 40, 40, 34, 34, 31],
              [84, 74, 69, 68, 63, 57, 56, 49, 41, 39, 32, 32],
              [84, 79, 79, 74, 74, 74, 72, 72, 59, 37, 37, 32],
              [84, 80, 79, 72, 66, 66, 64, 56, 50, 45, 43, 41],
              [84, 76, 52, 49, 47, 46, 40, 39, 37, 36, 34, 30],
              [82, 80, 79, 78, 75, 71, 64, 60, 45, 43, 35, 31],
              [84, 81, 81, 79, 73, 71, 61, 60, 60, 40, 31, 30],
              [83, 79, 78, 76, 72, 66, 57, 54, 53, 49, 47, 45],
              [83, 78, 76, 74, 73, 62, 57, 56, 42, 35, 32, 31],
              [84, 83, 82, 82, 64, 63, 54, 44, 39, 39, 35, 31],
              [81, 80, 72, 72, 70, 61, 55, 52, 42, 42, 38, 34],
              [81, 81, 72, 62, 59, 59, 51, 51, 50, 47, 34, 30],
              [84, 82, 79, 77, 76, 73, 67, 63, 60, 56, 50, 48],
              [77, 77, 70, 69, 68, 67, 65, 62, 54, 41, 34, 30],
              [80, 76, 74, 71, 71, 69, 65, 46, 44, 39, 36, 32],
              [84, 83, 81, 76, 75, 68, 64, 63, 53, 48, 36, 30],
              [74, 74, 69, 68, 62, 61, 58, 53, 44, 41, 36, 31],
              [80, 77, 71, 69, 63, 60, 59, 53, 52, 51, 44, 40]])

# Ganancia obtenida a fin de año por adquirir cada una de las empresas
# (cada fila le corresponde a una empresa)
g = np.array([[132, 130, 121, 112, 111,  96,  89,  88,  86,  80,  70,  61],
              [132, 123, 122, 113, 113, 106, 106,  70,  66,  64,  61,  61],
              [127, 115, 114, 107, 101,  79,  78,  78,  68,  68,  65,  64],
              [133, 129, 103, 100,  82,  78,  74,  70,  69,  64,  63,  60],
              [127, 122, 117, 112, 109, 108, 108, 105,  83,  77,  72,  71],
              [131, 118, 105,  99,  97,  89,  81,  80,  76,  69,  66,  62],
              [121, 120, 117, 115, 108, 102,  98,  93,  90,  84,  71,  60],
              [129, 127, 127, 125, 122, 120,  98,  96,  95,  90,  73,  60],
              [122, 121, 118, 114, 104,  98,  87,  65,  63,  63,  60,  60],
              [127, 121, 111, 104, 103,  94,  91,  89,  88,  82,  77,  74],
              [134, 131, 123, 111,  94,  92,  85,  81,  80,  67,  63,  60],
              [131, 119, 115, 109, 101,  95,  92,  87,  78,  77,  66,  60],
              [133, 132, 121, 115, 112, 105,  91,  89,  88,  81,  74,  69],
              [124, 123, 115, 113, 112, 111, 100,  97,  80,  73,  66,  65],
              [128, 126, 103, 102,  88,  81,  79,  75,  68,  64,  64,  62],
              [132, 119, 112, 112, 109, 109,  98,  95,  81,  80,  66,  61],
              [132,  95,  93,  88,  82,  80,  80,  78,  67,  66,  61,  61],
              [134, 131, 118, 116, 110, 102, 101,  95,  91,  89,  78,  71],
              [130, 128, 121, 120, 113, 103,  94,  87,  82,  81,  73,  73],
              [124, 115, 112, 109,  94,  92,  86,  83,  79,  74,  71,  61]])

In [ ]:
model = Model('MicroApple')

Observar que en este modelo tenemos muchas variables ($N\times 12$) así que definirlas a mano no es práctico. Utilizaremos una matriz de variables ``x`` de tamaño $N\times 12$ tal que en ``x[i, k]`` se guarde la variable $x_{ik}$.


In [ ]:
# Inicializamos la matriz vacía
x = np.empty((N, 12), dtype='object')

# La vamos completando con las variables
for i in range(???):
  for k in range(???):
    x[i, k] = model.addVar(f'x_{i}_{k}', vtype=???)


También utilizamos loops para agregar las restricciones

In [ ]:
# Comprar cada empresa a lo sumo una vez
for i in range(???):
  model.addCons(sum(x[i, k] for k in range(???)) <= 1)

# Respetar el presupuesto mensual
for k in range(???):
  model.addCons(sum(p[i, k] * x[i, k] for i in range(???)) <= ???)

# Reglamentacion antimonopolio
for s in range(???):
  model.addCons(sum(x[i, k] for ??? in ??? for ??? in ???) <= ???)

# Comprar a lo sumo una de las empresas 5, 7, 11
???

Agregamos la funcion objetivo

In [ ]:
model.setObjective(???, sense=???)

Resolvemos el modelo y reportamos la solucion

In [ ]:
model.redirectOutput()
model.optimize()

In [ ]:
sol = model.getBestSol()
model.writeSol(sol, 'solucion_MicroApple.sol')